<a href="https://colab.research.google.com/github/deepavasanthkumar/spark_delta_lake/blob/main/delta_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#QuickStart to DeltaLake

**Delta Lake** is an open source storage layer that brings reliability to data lakes. Delta Lake provides ACID transactions, scalable metadata handling, and unifies streaming and batch data processing. Delta Lake runs on top of your existing data lake and is fully compatible with Apache Spark APIs. Delta Lake on Databricks allows you to configure Delta Lake based on your workload patterns.

In [1]:
!pip install pyspark==3.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.5 MB 35 kB/s 
     |████████████████████████████████| 199 kB 48.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.2-py2.py3-none-any.whl size=281969454 sha256=621766735828552d66e61b3c173282b9a721daf73e08bc1399c917db25b6bf4b
  Stored in directory: /root/.cache/pip/wheels/f5/e6/d7/5216dc9246deb38346ab099a7f069df40f684fcd5968f44c0e
Successfully built pyspark


use pip install delta-spark and after successfull execution, **restart** runtime





In [2]:
!pip install delta-spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from delta.tables import DeltaTable
import shutil

In [ ]:
shutil.rmtree("/tmp/delta-table", ignore_errors=True)

#Creating SparkSession with ***configure_spark_with_delta_pip***



In [3]:
import pyspark
from delta import *
builder = SparkSession.builder.appName("DeltaLakeApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.jars.packages","io.delta:delta-core_2.12:2.0.0")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

#Write Data

In [4]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

#Read Data

In [5]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



#Upsert (merge) new data

In [ ]:
newData = spark.range(0, 20)

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

deltaTable.alias("oldData")\
    .merge(
    newData.alias("newData"),
    "oldData.id = newData.id")\
    .whenMatchedUpdate(set={"id": col("newData.id")})\
    .whenNotMatchedInsert(values={"id": col("newData.id")})\
    .execute()

deltaTable.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



#Update the Table

In [ ]:

data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")
deltaTable.toDF().show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
|  5|
|  6|
+---+



# Update every even value by adding 100 to it

In [ ]:
deltaTable.update(
    condition=expr("id % 2 == 0"),
    set={"id": expr("id + 100")})

deltaTable.toDF().show()

+---+
| id|
+---+
|  7|
|108|
|  9|
|  5|
|106|
+---+



# Delete every **even** value

In [ ]:
deltaTable.delete(condition=expr("id % 2 == 0"))
deltaTable.toDF().show()


+---+
| id|
+---+
|  7|
|  9|
|  5|
+---+



# Read old version of data using time travel

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



# cleanup

In [ ]:
shutil.rmtree("/tmp/delta-table")

In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7

#References




*   https://delta.io/learn/getting-started
*   https://github.com/delta-io/delta






#Issues Faced

I was trying to setup with pip install delta-spark
pip install deltalake

adding jars package with delta.io and all these were throwing error 



```
Caused by: java.lang.ClassNotFoundException: delta.DefaultSource
at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
```


This was happening when trying to use the delta format in write/read. 

`data.write.format("delta").save("/tmp/delta-table")`

This is internally referring to **tables.py** which is available in the github delta\python folder. Though there were multiple suggestions in stackoverflow, none of them, except manually copying the delta folder to python site packages worked.


This post helped to make the manual changes. 

https://stackoverflow.com/questions/65553722/no-module-named-delta-tables








In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7